# 1. Introduction to Image Segmentation

## 1.1 What is Image Segmentation?

### Definition
Pixel-wise classification: Assigning each pixel to a specific class or category

### Region-based separation
Dividing an image into distinct regions or objects

## 1.2 Purpose of Image Segmentation

- **Object detection**: Identifying and locating objects within an image
- **Measurement**: Computing area, perimeter, and shape characteristics
- **Medical imaging**: Analyzing anatomical structures and disease regions
- **Industrial inspection**: Quality control and defect detection
- **Robotics & autonomous systems**: Scene understanding and navigation
- **Preprocessing for feature extraction**: Preparing data for further analysis

## 1.3 Types of Segmentation

- **Threshold-based**: Using intensity values to separate foreground and background
- **Edge-based**: Detecting boundaries between regions
- **Region-based**: Growing regions from seed points or merging similar regions
- **Clustering-based**: Grouping pixels based on similarity (e.g., K-means)
- **Deep learning-based**: Using neural networks for pixel-wise classification (brief mention)

# 2. Thresholding Techniques

## 2.1 Basic Concept

### Convert grayscale image → binary image
Thresholding is the simplest form of segmentation that converts a grayscale image into a binary image.

### Threshold value $T$
A specific intensity value that separates the foreground from the background.

### Binary rule

$$g(x,y) = \begin{cases} 255 & \text{if } f(x,y) > T \\ 0 & \text{otherwise} \end{cases}$$

Where:
- $f(x,y)$ is the input grayscale image
- $g(x,y)$ is the output binary image
- $T$ is the threshold value

## 2.2 Global Thresholding

### Theory

- **Single threshold for entire image**: One threshold value $T$ is applied uniformly across the entire image
- **Works when lighting is uniform**: Most effective when image has consistent illumination
- **OpenCV Function**:
```python
ret, thresh = cv2.threshold(img, T, 255, cv2.THRESH_BINARY)
```

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# Load images from img folder
img_folder = Path('img')

# Load a color image and convert to grayscale
img_color = cv2.imread(str(img_folder / 'cleon.jpg'), cv2.IMREAD_GRAYSCALE)

# Create figure for histogram and thresholding demonstration
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

# Original image
axes[0].imshow(img_color, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Histogram
axes[1].hist(img_color.flatten(), 256, [0, 256], color='black')
axes[1].set_title('Histogram')
axes[1].set_xlabel('Pixel Intensity')
axes[1].set_ylabel('Frequency')

# Apply different threshold values
threshold_values = [50, 128, 200]
for idx, T in enumerate(threshold_values):
    ret, thresh = cv2.threshold(img_color, T, 255, cv2.THRESH_BINARY)
    axes[idx + 2].imshow(thresh, cmap='gray')
    axes[idx + 2].set_title(f'T = {T}')
    axes[idx + 2].axis('off')

plt.tight_layout()
plt.show()


## 2.3 Adaptive Thresholding

### Theory

- **Different threshold for different regions**: Instead of a global $T$, the threshold varies across the image
- **Useful under non-uniform illumination**: Handles images with varying lighting conditions effectively

### Types

- **Mean**: Threshold is the mean of the neighborhood area
- **Gaussian**: Threshold is a weighted sum using a Gaussian window

### OpenCV Function

```python
cv2.adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, C)
```

### Parameters Explanation

- **Block size**: Size of the neighborhood area ($blockSize × blockSize$). Must be odd (e.g., 3, 5, 7, 11, etc.)
- **C value**: Constant subtracted from the mean/weighted mean
  - Positive $C$: Makes thresholding less strict (more white pixels)
  - Negative $C$: Makes thresholding more strict (more black pixels)
- **Method type**:
  - `cv2.ADAPTIVE_THRESH_MEAN_C`: Mean of the neighborhood area
  - `cv2.ADAPTIVE_THRESH_GAUSSIAN_C`: Gaussian-weighted sum of the neighborhood

### Comparison: Global vs Adaptive

Visual comparison showing when global thresholding fails and adaptive thresholding succeeds.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# Load image with non-uniform lighting
img_folder = Path('img')
img = cv2.imread(str(img_folder / 'cleon.jpg'), cv2.IMREAD_GRAYSCALE)

# Create figure for comparison
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Row 1: Global Thresholding
axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Global thresholding with different T values
global_thresholds = [100, 128, 150]
for idx, T in enumerate(global_thresholds):
    ret, thresh_global = cv2.threshold(img, T, 255, cv2.THRESH_BINARY)
    axes[0, idx + 1].imshow(thresh_global, cmap='gray')
    axes[0, idx + 1].set_title(f'Global T={T}', fontsize=12)
    axes[0, idx + 1].axis('off')

# Row 2: Adaptive Thresholding
axes[1, 0].imshow(img, cmap='gray')
axes[1, 0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

# Adaptive thresholding with different block sizes and methods
configs = [
    (11, 2, cv2.ADAPTIVE_THRESH_MEAN_C, 'Mean\nBlockSize=11, C=2'),
    (15, 2, cv2.ADAPTIVE_THRESH_MEAN_C, 'Mean\nBlockSize=15, C=2'),
    (21, 5, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 'Gaussian\nBlockSize=21, C=5')
]

for idx, (blockSize, C, method, title) in enumerate(configs):
    thresh_adaptive = cv2.adaptiveThreshold(img, 255, method, cv2.THRESH_BINARY, blockSize, C)
    axes[1, idx + 1].imshow(thresh_adaptive, cmap='gray')
    axes[1, idx + 1].set_title(title, fontsize=12)
    axes[1, idx + 1].axis('off')

# Set row labels
fig.text(0.02, 0.75, 'Global\nThresholding', va='center', fontsize=12, fontweight='bold', rotation=0)
fig.text(0.02, 0.25, 'Adaptive\nThresholding', va='center', fontsize=12, fontweight='bold', rotation=0)

plt.suptitle('Global vs Adaptive Thresholding Comparison', fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout(rect=[0.05, 0, 1, 0.96])
plt.show()




## Comparison: Global vs Adaptive Thresholding

| Aspect | Global Thresholding | Adaptive Thresholding |
|--------|-------------------|----------------------|
| **Threshold Value** | Single value $T$ applied to entire image | Different values $T(x,y)$ for each region |
| **Computation** | `ret, thresh = cv2.threshold(img, T, 255, cv2.THRESH_BINARY)` | `cv2.adaptiveThreshold(img, 255, method, cv2.THRESH_BINARY, blockSize, C)` |
| **Illumination Handling** | Fails with non-uniform lighting | Handles non-uniform lighting well |
| **Parameters** | Only threshold value $T$ | Block size, C value, method (Mean/Gaussian) |
| **Speed** | Very fast (single pass) | Slower due to neighborhood calculations |
| **Use Cases** | Images with uniform lighting, well-separated objects | Scanned documents, medical images, varying lighting |
| **Effect of Lighting Changes** | Threshold becomes invalid in darker/brighter regions | Automatically adapts to local conditions |
| **Detail Preservation** | May lose detail in shadow or highlight regions | Better detail preservation across entire image |
| **Parameter Tuning Ease** | Simple - choose one value $T$ | Requires tuning: block size and C value |

### When to Use Each Method

**Use Global Thresholding when:**
- Image has uniform, consistent lighting
- You need maximum speed
- Objects are clearly separated with distinct intensity values
- Simple threshold value works across entire image

**Use Adaptive Thresholding when:**
- Image has non-uniform lighting or shadows
- Processing scanned documents or photographs
- Objects have similar intensity but different local context
- Detail preservation is critical

## 2.4 Otsu's Thresholding

### Theory

- **Automatic threshold selection**: Otsu's method automatically determines the optimal threshold value
- **Minimizes intra-class variance**: Finds the threshold that minimizes variance within each class (foreground and background)
- **Uses histogram statistics**: Based on the distribution of pixel intensities in the image

### Mathematical Idea (Brief)

**Between-class variance maximization**: 
Otsu's method finds the threshold $T$ that maximizes the between-class variance:

$$\sigma_B^2(T) = \omega_0(T) \omega_1(T) [\mu_0(T) - \mu_1(T)]^2$$

Where:
- $\omega_0(T)$ = weight of background class (proportion of pixels below $T$)
- $\omega_1(T)$ = weight of foreground class (proportion of pixels above $T$)
- $\mu_0(T)$ = mean intensity of background pixels
- $\mu_1(T)$ = mean intensity of foreground pixels

The optimal threshold $T^*$ maximizes this variance.

### OpenCV Function

```python
ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
```

Note: The threshold value (0) is ignored; `ret` returns the automatically calculated optimal threshold.

### Why Otsu Works Well

 **Bimodal distributions**: Otsu's method works best when the histogram has two distinct peaks (one for background, one for foreground)

 **Poor performance**: May fail on images with uniform or multi-modal histograms

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# Load image
img_folder = Path('img')
img = cv2.imread(str(img_folder / 'cleon.jpg'), cv2.IMREAD_GRAYSCALE)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Original image
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Histogram with Otsu threshold line
hist = cv2.calcHist([img], [0], None, [256], [0, 256])
axes[1].plot(hist, color='black', linewidth=1)
otsu_threshold, _ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
axes[1].axvline(otsu_threshold, color='red', linewidth=2, linestyle='--', label=f'Otsu T = {otsu_threshold}')
axes[1].set_title('Histogram', fontsize=12)
axes[1].set_xlabel('Pixel Intensity')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Result of Otsu thresholding
_, thresh_otsu = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
axes[2].imshow(thresh_otsu, cmap='gray')
axes[2].set_title(f"Otsu's Result\n(T = {otsu_threshold})", fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.suptitle("Otsu's Thresholding", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## Comprehensive Comparison: Global vs Adaptive vs Otsu's Thresholding

| Aspect | Global | Adaptive | Otsu's |
|--------|--------|----------|--------|
| **Threshold Selection** | Manual - user specifies T | Automatic - computed per neighborhood | Automatic - computed once globally |
| **Number of Thresholds** | Single value $T$ | Multiple values $T(x,y)$ per region | Single optimal value $T^*$ |
| **OpenCV Function** | `cv2.threshold(img, T, 255, cv2.THRESH_BINARY)` | `cv2.adaptiveThreshold(img, 255, method, cv2.THRESH_BINARY, blockSize, C)` | `cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)` |
| **Computation Speed** | Very Fast | Slow (neighborhood analysis) | Fast (histogram analysis) |
| **Best For** | Uniform lighting, simple images | Non-uniform lighting, documents, photos | Bimodal histogram, automated processing |
| **Illumination Robustness** |Poor with shadows/varying light | Excellent - adapts locally | Depends on histogram shape |
| **Parameter Tuning** | Simple (1 parameter: T) | Complex (blockSize, C, method) | None (fully automatic) |
| **Histogram Requirement** | Works with any distribution | No specific requirement | Best with bimodal (two peaks) |
| **Failed Case** | Non-uniform lighting, shadows | Flat/uniform images | Multi-modal histogram, uniform images |
| **Computational Cost** | $O(n)$ where n = pixels | $O(n \times blockSize^2)$ | $O(n + 256)$ |
| **Detail Preservation** | May lose detail in dark/bright areas | Excellent detail across entire image | Good when histogram is bimodal |
| **Manual Adjustment** | Flexible - can tweak T easily | Requires re-tuning blockSize & C | Not possible - fully automatic |

### Decision Guide: Which Method to Use?

**Choose Global Thresholding if:**
- ✓ Image has uniform, consistent lighting
- ✓ You want maximum speed and simplicity
- ✓ Objects have clearly separated intensity values
- ✓ You're processing real-time applications (video, streaming)
- ✓ You need manual control over the threshold

**Choose Adaptive Thresholding if:**
- ✓ Image has non-uniform lighting or shadows
- ✓ Processing scanned documents, photographs, or medical images
- ✓ Objects with similar intensity need local context discrimination
- ✓ Detail preservation in all regions is critical
- ✓ You can afford moderate computational cost
- ✓ You have time to tune blockSize and C parameters

**Choose Otsu's Thresholding if:**
- ✓ Need fully automatic threshold selection (no manual tuning)
- ✓ Image histogram has clear bimodal distribution
- ✓ Want fast processing with reasonable quality
- ✓ Processing large batches of similar images (e.g., document scanning)
- ✓ You don't have time for parameter tuning
- ✗ Image histogram is multimodal or flat (this method will fail)

### Summary Table: Pros and Cons

| Method | Pros | Cons |
|--------|------|------|
| **Global** | Fast, simple, flexible | Fails with shadows, needs manual tuning, uniform lighting only |
| **Adaptive** | Handles varying light, preserves detail, robust | Slow, complex tuning, may fail on uniform images |
| **Otsu's** | Automatic, fast, no tuning needed | Only for bimodal histograms, fails on multi-modal/uniform |